In [ ]:
import random, csv
from pathlib import Path
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt

SEED = 1337
random.seed(SEED)

ROOT = Path(__file__).resolve().parents[2]
RESULTS_DIR = ROOT / "results" / date.today().isoformat() / "stress_firebreak"
FIG_DIR = ROOT / "fig"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

rows = []
for _ in range(96):
    k = random.choice([2,3,4])
    n = random.choice([2**p for p in range(6, 14)])
    d = random.choice([1,2,3])
    extra = random.choice([1,2,4])
    pass_region = (d <= 3) and (extra == 1)
    rows.append({"k": k, "n": n, "d": d, "extra": extra, "pass": int(pass_region)})

df = pd.DataFrame(rows)
out_csv = RESULTS_DIR / "stress_matrix.csv"
df.to_csv(out_csv, index=False)
print(f"wrote {out_csv}")



In [ ]:
import matplotlib.pyplot as plt

# Aggregate pass rate by k and extra budget
pivot = df.pivot_table(values="pass", index="k", columns="extra", aggfunc="mean")
ax = pivot.plot(kind="bar", rot=0, title="Pass rate by k and extra budget (seed=1337)")
ax.set_ylabel("pass rate")
fig_path = FIG_DIR / "stress_firebreak_overview.png"
plt.tight_layout()
plt.savefig(fig_path, dpi=160)
print(f"saved {fig_path}")

